In [1]:
#Importing libraries required
import numpy as np
import random
import re
from nltk.util import ngrams
import itertools
import pandas as pd
from sklearn import svm 
from joblib import dump,load
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split